In [6]:
from temporaldata import Data
import h5py
import os
import pandas as pd
data_folder = "../../../../data/processed/allen_visual_coding_ophys_2016"
output_csv = "task_contents.csv"


tasks = set()
canonical_tasks = [
    "pupil.location",
    "running.running_speed",
    "drifting_gratings.orientation",
    "drifting_gratings.temporal_frequency",
    "static_gratings.phase",
    "static_gratings.spatial_frequency",
    "static_gratings.orientation",
    "natural_scenes.frame",
    "natural_movie_one.frame",
    "natural_movie_two.frame",
    "natural_movie_three.frame",
    "locally_sparse_noise.frame",
]
df = pd.DataFrame()
for session_id in os.listdir(data_folder):
    session_path = os.path.join(data_folder, session_id)
    with h5py.File(session_path, "r") as f:
        data = Data.from_hdf5(f)
        data.materialize()
    
    row = {
        "session_id": session_id,
    }
    for task in canonical_tasks:
        task_name, value_name = task.split(".")
        if hasattr(data, task_name):
            if hasattr(getattr(data, task_name), value_name):
                row[task] = True
            else:
                row[task] = False
        else:
            row[task] = False
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

df.to_csv(output_csv, index=False)
        